# How to deal with json documents

Load data from JSON formatted document into the index


In [1]:
import os
import json
import base64
import pandas as pd



from langchain_openai import AzureOpenAIEmbeddings


from dotenv import load_dotenv
load_dotenv("../apps/credentials.env")

def text_to_base64(text):
    # Convert text to bytes using UTF-8 encoding
    bytes_data = text.encode('utf-8')

    # Perform Base64 encoding
    base64_encoded = base64.b64encode(bytes_data)

    # Convert the result back to a UTF-8 string representation
    base64_text = base64_encoded.decode('utf-8')

    return base64_text

In [2]:

# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

In [3]:
embedder = AzureOpenAIEmbeddings(deployment=os.environ["EMBEDDING_DEPLOYMENT_NAME"], chunk_size=1)

## Create Vector-based index


Now that we have the content of the book's chunks (each page of each book) in the dictionary `book_pages_map`, let's create the Vector index in our Azure Search Engine where this content is going to land

In [ ]:
index_name= os.environ["AZURE_SEARCH_INDEX_NAME"]
print("Index name: ", index_name)

In [5]:
### Create Azure Search Vector-based Index
# Setup the Payloads header
headers = {'Content-Type': 'application/json','api-key': os.environ['AZURE_SEARCH_KEY']}
params = {'api-version': os.environ['AZURE_SEARCH_API_VERSION']}

REST API version 2023-10-01-Preview supports external and internal vectorization. This Notebook assumes an external vectorization strategy. This API also supports:
    
- vectorSearch algorithms, hnsw and exhaustiveKnn nearest neighbors, with parameters for indexing and scoring.
- vectorProfiles for multiple combinations of algorithm configurations.

Vector search algorithms include **exhaustive k-nearest neighbors (KNN)** and **Hierarchical Navigable Small World (HNSW)**. Exhaustive KNN performs a brute-force search that scans the entire vector space. HNSW performs an approximate nearest neighbor (ANN) search. While KNN provides exact nearest neighbor search results with high accuracy, its computational cost and poor scalability make it impractical for large datasets or real-time applications. HNSW, on the other hand, offers a highly efficient and scalable solution for nearest neighbor searches by finding approximate nearest neighbors quickly, making it more suitable for large-scale and high-dimensional data applications.


check [HERE](https://learn.microsoft.com/en-us/azure/search/vector-search-how-to-create-index?tabs=config-2023-10-01-Preview%2Crest-2023-11-01%2Cpush%2Cportal-check-index) for the details of the vector configuration.

In [ ]:
import requests

index_payload = {
    "name": index_name,
    "vectorSearch": {
        "algorithms": [  # We are showing here 3 types of search algorithms configurations that you can do
             {
                 "name": "my-hnsw-config-1",
                 "kind": "hnsw",
                 "hnswParameters": {
                     "m": 4,
                     "efConstruction": 400,
                     "efSearch": 500,
                     "metric": "cosine"
                 }
             },
             {
                 "name": "my-hnsw-config-2",
                 "kind": "hnsw",
                 "hnswParameters": {
                     "m": 8,
                     "efConstruction": 800,
                     "efSearch": 800,
                     "metric": "cosine"
                 }
             },
             {
                 "name": "my-eknn-config",
                 "kind": "exhaustiveKnn",
                 "exhaustiveKnnParameters": {
                     "metric": "cosine"
                 }
             }
        ],
        "vectorizers": [
            {
                "name": "openai",
                "kind": "azureOpenAI",
                "azureOpenAIParameters":
                {
                    "resourceUri" : os.environ['AZURE_OPENAI_ENDPOINT'],
                    "apiKey" : os.environ['AZURE_OPENAI_API_KEY'],
                    "deploymentId" : "text-embedding-ada-002",
                    "modelName": "text-embedding-ada-002"
                }
            }
        ],
        "profiles": [  # profiles is the diferent kind of combinations of algos and vectorizers
            {
             "name": "my-vector-profile-1",
             "algorithm": "my-hnsw-config-1",
             "vectorizer":"openai"
            },
            {
             "name": "my-vector-profile-2",
             "algorithm": "my-hnsw-config-2",
             "vectorizer":"openai"
            },
            {
             "name": "my-vector-profile-3",
             "algorithm": "my-eknn-config",
             "vectorizer":"openai"
            }
        ]
    },
    "semantic": {
        "configurations": [
            {
                "name": "my-semantic-config",
                "prioritizedFields": {
                    "titleField": {
                        "fieldName": "title"
                    },
                    "prioritizedContentFields": [
                        {
                            "fieldName": "chunk"
                        }
                    ],
                    "prioritizedKeywordsFields": []
                }
            }
        ]
    },
    "fields": [
        {"name": "id", "type": "Edm.String", "key": "true", "filterable": "true" },
        {"name": "title","type": "Edm.String","searchable": "true","retrievable": "true"},
        {"name": "chunk","type": "Edm.String","searchable": "true","retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "location", "type": "Edm.String", "searchable": "false", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {
            "name": "chunkVector",
            "type": "Collection(Edm.Single)",
            "dimensions": 1536,
            "vectorSearchProfile": "my-vector-profile-3", # we picked profile 3 to show that this index uses eKNN vs HNSW (on prior notebooks)
            "searchable": "true",
            "retrievable": "true",
            "filterable": "false",
            "sortable": "false",
            "facetable": "false"
        }

    ],
}
url=f"{os.environ['AZURE_SEARCH_ENDPOINT']}/indexes/{index_name}"
print(url)
r = requests.put(url,
                 data=json.dumps(index_payload), headers=headers, params=params)
print(r.status_code)
print(r.text)
print(r.ok)

## Upload the Document chunks and its vectors to the Index

The following code will iterate over each chunk of each book and use the Azure Search Rest API upload method to insert each document with its corresponding vector (using OpenAI embedding model) to the index.

In [7]:
#load data from jsonl file
data_file='data.jsonl'
data = pd.read_json(data_file, lines=True)

In [ ]:
#iterarte over dataframe and access each column's value to populate index

for index, row in data.iterrows():
    title=row['title']
    keywords=row['keywords']
    summary=row['summary']
    content=row['content']
    source_url=row['source_url']
    chunk=f"""
    title: {title}
    keywords: {keywords}
    summary: {summary}
    content: {content}
    source url: {source_url}
    """
    # Create the payload
    payload = {
        "value": [
            {
                "@search.action": "upload",
                "id": str(index),
                "title": title,
                "chunk": chunk,
                "location": source_url,
                "chunkVector": embedder.embed_query(chunk if chunk!="" else "-------")
            }
        ]
    }
    try:
        r = requests.post(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + index_name + "/docs/index",
                    data=json.dumps(payload), headers=headers, params=params)
        print(f"added article {id} to index")
        if r.status_code != 200:
            print(r.status_code)
            print(r.text)
    except Exception as e:
        print("Exception:",e)
        print(chunk)
        continue
